In [1]:
import torch 
from torch import nn 
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from dataset import get_dataloaders
from model import ParametricModel

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device

'cuda'

In [3]:
train_loader, test_loder = get_dataloaders(csv_path='xy_data.csv')
model = ParametricModel().to(device)

In [4]:
loss_fn = nn.L1Loss()
optimizer = Adam(params=model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=10) 

In [5]:
num_epochs = 10000 
print_freq = 200

In [6]:
for epoch in range(num_epochs):
    
    # train 
    model.train()
    for batch in train_loader:

        t = batch['t'].to(device)
        x = batch['x'].to(device)
        y = batch['y'].to(device)
        x_pred, y_pred = model(t)
        train_loss1 = loss_fn(x, x_pred)
        train_loss2 = loss_fn(y, y_pred)
        train_loss = train_loss1 + train_loss2 

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    # test
    model.eval()
    with torch.no_grad():
        for batch in test_loder:
            
            t = batch['t'].to(device)
            x = batch['x'].to(device)
            y = batch['y'].to(device)

            test_loss1 = loss_fn(x, x_pred)
            test_loss2 = loss_fn(y, y_pred)
            test_loss = test_loss1 + test_loss2 

    scheduler.step(test_loss)
    if epoch % print_freq == 0:
        #model.print_params()
        print(f"Loss: {test_loss}")
               

Loss: 31.223581314086914
Loss: 24.296260833740234
Loss: 26.937223434448242
Loss: 23.688453674316406
Loss: 21.47234344482422
Loss: 20.231855392456055
Loss: 23.89421844482422
Loss: 23.62628936767578
Loss: 22.955963134765625
Loss: 23.031906127929688
Loss: 25.252866744995117
Loss: 20.448162078857422
Loss: 21.479358673095703
Loss: 21.74849510192871
Loss: 23.24258804321289
Loss: 20.711692810058594
Loss: 27.386152267456055
Loss: 25.100543975830078
Loss: 23.337982177734375
Loss: 22.22559356689453
Loss: 24.06841468811035
Loss: 21.95918846130371
Loss: 22.434734344482422
Loss: 19.315536499023438


KeyboardInterrupt: 

In [12]:
print(f'Theta: {model.theta}\nM: {model.M}\nX: {model.X}')

Theta: Parameter containing:
tensor(25.6154, device='cuda:0', requires_grad=True)
M: Parameter containing:
tensor(0.0186, device='cuda:0', requires_grad=True)
X: Parameter containing:
tensor(53.5555, device='cuda:0', requires_grad=True)


In [13]:
# 0° < theta < 50°
# -0.05 < M < 0.05
# 0 < X < 100